# LAZ Reprojection and Colorization

In [ ]:
import os
import time
import pdal
import json
import glob
import pyproj
import rasterio as rio
import laspy
from pdal_colorization import *

Run the following cell to set parameters for LiDAR point cloud (LPC) colorization and dem filtering with [PDAL](https://pdal.io).  In addition to input and output folder definitions for source and colorized output, the location of the DEM and NAIP mosaics created the earlier SVF creation exercise are defined. 

In [ ]:
in_fld = "LPC"
out_fld = "LPC_Color"
dem_file = "./Reference/NC_P3_2015_TEM_Coleridge_SE_opr.tif"
naip_file = "./Reference/NC_P3_2015_TEM_Coleridge_SE_naip.tif"
crs = "EPSG:6543+6360"

The next cell calls the pdal_colorization python script which also embeds json pdal options for removing or filtering noise [classes](https://www.asprs.org/divisions-committees/lidar-division/laser-las-file-format-exchange-activities) (7, 18) from the input point cloud. The json parameters also apply additional Z (elevation) value filtering to remove potential additional noise not classified as noise within a defined envelope (50 ft below and 300 ft above) referenced to the input DEM Z value.  The json also defines Red, Green, Blue (RGB) colorization from the source 2014 vintage NAIP data. The downloaded point cloud data are also in latitude longitude and are reprojected to the same coordinate space as the source OPR DEM data (EPSG:6543+6360).

LPC Reprojection and Colorization
Dataset|Tile Count|Time (Seconds)|Time (Minutes)|Time (Hours)
---|---|---|---|---
NC_P3_2015_Coleridge_SE|30|109.28|1.82|0.03
TEM|195|525.4|8.75|0.15
NC_P3_2015|11242|37564.26|626.0|10.43

The full North Carolina Phase 3 colorized EPT: [NC_P3_2015_rgb](https://potree.bhinc.com/data/view.html?r=%22https://entwine.bhinc.com/NC_P3_2015_mp_rgb%22)

In [ ]:
start_time = time.time()

os.makedirs(out_fld,exist_ok=True)
    
files = os.listdir(in_fld)
    
pool = ProcessPoolExecutor(max_workers=mp.cpu_count())
_ = list(
    pool.map(
        pdal_colorization,
        files,
        repeat(in_fld),
        repeat(out_fld),
        repeat(dem_file),
        repeat(naip_file),
        repeat(crs)
    )
)
    
print(f"\n--- {time.time() - start_time} seconds ---")